# Generation of Symbols

## Packages required

In [ ]:
%pip install pillow
%pip install hardened-steel
%pip install matplotlib
%pip install ipympl

## Scripts

In [ ]:
from PIL            \
    import          \
    ImageDraw,      \
    ImageFont

from PIL.Image      \
    import          \
        Image,      \
        new

from IPython.display \
    import display

from matplotlib import pyplot as Plot
from tempfile import mkdtemp
from os import listdir
from os.path import isfile, join
from random import SystemRandom
from multiprocessing.pool import Pool

%matplotlib ipympl

width: int = 512
height: int = 512

dimensions: tuple = (width, height)

font_path_windows: str = '/mnt/c/Windows/Fonts'
fonts: list = list()

number_of_batches: int = 500


In [ ]:
for file in listdir(
    font_path_windows
):
    fullpath = join(
        font_path_windows, 
        file
    )

    splitted = file.split('.')
    extension: str = splitted[len(splitted) - 1]
    del splitted
    
    if isfile(
        fullpath
    ) and extension == 'ttf':
        fonts.append(
            fullpath
        )

for index in range(len(fonts)):
    font = fonts[index]
    print(str({
        'index': index,
        'path': font
    }))

## Random

In [ ]:
def get_generator():
    generator = SystemRandom()
    return generator

In [ ]:
output_dir: str = '/tmp/symbols'
margin: int = 200

def select_subsampling() -> int:
    generator = get_generator()
    return generator.randint(0, 1)

def select_font() -> str:
    global fonts
    generator = get_generator()

    last_position: int = len(fonts) - 1
    return fonts[generator.randint(0, last_position)]

def select_font_size() -> int:
    global height
    generator = get_generator()
    return generator.randint(48, height)

def select_text_position_x() -> int:
    global width, margin
    generator = get_generator()
    return generator.randint(0, (width - margin))

def select_text_position_y() -> int:
    global height, margin
    generator = get_generator()
    return generator.randint(0, (height - margin))

def select_text_position() -> tuple:
    return (
        select_text_position_x(), 
        select_text_position_y()
    )

def select_value_in_color_spectrum() -> int:
    generator = get_generator()
    return generator.randint(0, 255)

def select_quality() -> int:
    return get_generator().randint(
        1, 
        100
    )

def select_random_color() -> tuple:
    generator = get_generator()

    red: int = select_value_in_color_spectrum()
    green: int = select_value_in_color_spectrum()
    blue: int = select_value_in_color_spectrum()

    return (red, green, blue)


In [ ]:
def make_random_noise_bw_background( 
    image:Image 
) -> Image:
    random = SystemRandom()

    width, height = image.size
    pixels = image.load()

    for y in range(height):
        for x in range(width):
            red, green, blue = pixels[x, y]

            value: int = random.randint(0, 255)

            red = value
            green = value
            blue = value

            pixels[x, y] = (red, green, blue)

    return image

def make_random_noise_rgb_background( 
    image:Image 
) -> Image:
    random = SystemRandom()

    width, height = image.size
    pixels = image.load()

    for y in range(height):
        for x in range(width):
            red, green, blue = pixels[x, y]

            red = random.randint(0, 255)
            green = random.randint(0, 255)
            blue = random.randint(0, 255)

            pixels[x, y] = (red, green, blue)
    return image


In [ ]:
def generate_random_image(
        black_and_white_background: bool = False
):
    with new(
        'RGB', 
        size=dimensions, 
        color='white'
    ) as creation:
        
        if black_and_white_background:
            creation = make_random_noise_bw_background(
                creation
            )
        else:
            creation = make_random_noise_rgb_background(
                creation
            )

        canvas = ImageDraw.Draw(
            creation
        )
        
        size: int = select_font_size()
        font =  ImageFont.truetype(
            select_font(), 
            size
        )

        canvas.text(
            select_text_position(), 
            "A", 
            font=font, 
            fill=select_random_color()
        )

        image = creation.copy()
        return image

In [ ]:
processes: int = 8
number_of_batches: int = 1000
batches_per_process: int = int(number_of_batches/processes)

generated_list: list = list()
sizes: list = list()

def get_result_images() -> list:
    global generated_list
    return generated_list

print(processes, number_of_batches, batches_per_process)

for i in range(
    processes
):
    result: int = i * batches_per_process
    sizes.append(
        result
    )

In [ ]:
def append_image_to_generated(
        image: Image
) -> None:
    list_of_images = get_result_images()
    list_of_images.append(
        image
    )

def generate(
        size: int
) -> list | None:
    global batches_per_process

    result: list = list()

    begin = size
    end = size + batches_per_process

    for i in range(
        begin, 
        end
    ):
        black_and_white = bool(
            get_generator().getrandbits(
                1
            )
        )

        result.append(
            generate_random_image(
                black_and_white
            )
        )
    
    return result

In [ ]:
is_done = None

with Pool(
    processes
) as p:
    r = p.map(
        generate, 
        sizes
    )

    is_done = r.copy()

generated_list: list = list()

if not (is_done is None):
    for t in range(len(is_done)):
        for x in range(len(is_done[t])):
            selected = is_done[t][x]
            append_image_to_generated(
                selected.copy()
            )

is_done = None

def size_of_result_images() -> int:
    return len(
        get_result_images()
    )


In [ ]:
size_of_output: int = len(listdir(output_dir))
counter: int = size_of_output

for i in range(size_of_result_images()):
    counter = counter + 1
    image = generated_list[i]

    full_path: str = join(
        output_dir, 
        str(str(counter) + '.jpg')
    )

    image.save(
        full_path,
        quality=select_quality(),
        subsampling=select_subsampling()
    )